In [18]:
import tools
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()
print(os.getenv("LANGSMITH_ENDPOINT"))

https://api.smith.langchain.com


In [19]:
import os

import os
from langchain.chat_models import init_chat_model



llm = init_chat_model(
    "azure_openai:gpt-4.1-mini",
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    api_version="2024-12-01-preview"
)



In [21]:
import json
base_prompt = '''
Antworte in JSON und folge dem Schema: {
  "type": "object",
  "properties": {
    "response": {
      "type": "string",
      "enum": ["yes", "no", "no answer"]
    },
    "reasoning": {
      "type": "string"
    }
  },
  "required": ["response", "reasoning"]
}
'''
with open("datasets/dataset_850.json", "r") as file:
    dataset = json.load(file)


responses = []
for data in dataset:
    sentence = data["sentence"]
    prompt = f"{sentence}\n{base_prompt}"
    
    try:
        response = llm.invoke(prompt)
    except Exception as e:
        responses.append({
            "id": data["id"],
            "plant": data["plant"],
            "city": data["city"],
            "date": data["date"],
            "sentence": sentence,
            "response": "error",
            "reasoning": str(e),
            "tokens": {
                "completion_tokens": 0,
                "prompt_tokens": 0,
                "total_tokens": 0
            }
        })
    cleaned_response = response.content.strip("```json").strip("```").strip()
    response_json = json.loads(cleaned_response)
    responses.append({
        "id": data["id"],
        "plant": data["plant"],
        "city": data["city"],
        "date": data["date"],
        "sentence": sentence,
        "response": response_json["response"],
        "reasoning": response_json["reasoning"],
        "tokens": {
            "completion_tokens": response.response_metadata["token_usage"]["completion_tokens"],
            "prompt_tokens": response.response_metadata["token_usage"]["prompt_tokens"],
            "total_tokens": response.response_metadata["token_usage"]["total_tokens"]
        }
    })

print(responses)

with open("results/only_llm_850.json", "w") as file:
    json.dump(responses, file, indent=4)



[{'id': 0, 'plant': 'Apfel', 'city': 'Nürnberg', 'date': '29.04.2021', 'sentence': 'Haben sich bis 29.04.2021 in Nürnberg bereits Blüten an der Apfel gezeigt?', 'response': 'no answer', 'reasoning': 'Es liegen keine spezifischen Daten zur Apfelblüte in Nürnberg bis zum 29.04.2021 vor. Die Apfelblüte beginnt typischerweise Ende April bis Anfang Mai, aber ohne genauere lokale Beobachtungen ist keine sichere Aussage möglich.', 'tokens': {'completion_tokens': 74, 'prompt_tokens': 110, 'total_tokens': 184}}, {'id': 1, 'plant': 'Apfel', 'city': 'Berlin', 'date': '17.05.2022', 'sentence': 'Hat die Apfel in Berlin bis zum 17.05.2022 schon mit der Blüte begonnen oder ist sie dieses Jahr bereits aufgeblüht?', 'response': 'yes', 'reasoning': 'In Berlin beginnt die Blütezeit der Apfelbäume in der Regel Ende April bis Anfang Mai. Bis zum 17.05.2022 sind die Apfelbäume in der Regel bereits aufgeblüht, da die Blüte normalerweise etwa 1-2 Wochen dauert und oft schon Mitte Mai abgeschlossen ist.', 'tok

In [ ]:
response.response_metadata["token_usage"]["completion_tokens"]
response.response_metadata["token_usage"]["prompt_tokens"]
response.response_metadata["token_usage"]["total_tokens"]
#json.loads(response.content.strip("```json").strip("```").strip())

198